# **Thermal Waves Extension**
This jupyter notebook contains the data analysis for the extension section of the script.


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from data_utils import load_dataset
from collections import defaultdict
import re
import os
from IPython.display import Math, display

## **Heat Diffusion through a Cylinder**

The flow of heat in a medium is described by the **heat equation**, which for a 1-D system) is:


$$
\frac{\partial T(x,t)}{\partial t} = D \frac{\partial^2 T(x,t)}{\partial x^2}
$$


where $T(x,t)$ is the temperature in the medium as a function of space and time, and $D$ is the **thermal diffusivity** of the medium.

---

### Analytical Solution
Using a trial solution of the form $T(x, t) = X(x)\,\theta(t)$ we can write a solution to this equation:

$$
T(x,t) = \left(C_{-}\,e^{\beta x} + C_{+}e^{-\beta x} \right)\,e^{-i\omega t}
$$
where $C_{-}$ and $C_{+}$ are complex amplitudes and $\beta$ is a complex propagation coefficient:
$$
\beta = (1+i)\sqrt{\frac{\omega}{2D}}
$$
We assume the top end of the cylinder ($x = L$) is non-conductive so:
$$
\left. \frac{\partial T}{\partial x} \right|_{x=L} = 0
$$

After applying this condition we can rewrite our solution in terms of a single complex amplitude $K$:

$$
T(x, t) = K \cosh\left( \beta(L-x) \right)
$$

---

### Geometry of the Experiment

In the plane slab model, the brass cylinder is treated as a **1-dimensional slab** of thickness

$$
x_i = i\,\Delta d
$$

with the x-axis along the cylinder’s axis.  

Thermal waves propagate along the x-axis, and the thermistor sensors are placed every $\Delta d = 5~\text{mm}$.

We define $x = 0$ at the first thermistor.


In [16]:
delta_d = 5  # 5 mm spacing between thermistors

In [17]:
save_dir = "Thermal Wave Data"  # The folder that contains the data

files = ['brass_T10s_run1.csv', 'brass_T10s_run3.csv',
         'brass_T15s_run1.csv', 'brass_T15s_run2.csv',
         'brass_T20s_run1.csv', 'brass_T20s_run2.csv', 'brass_T20s_run3.csv',
         'brass_T30s_run1.csv', 'brass_T30s_run2.csv',
         'brass_T60s_run1.csv', 'brass_T60s_run2.csv']

pattern = re.compile(r"T(\d+)s_run(\d+)")  # Name format to extract time period and run number

### **Manually Truncate Datasets to Remove Transients**

First we will plot all datasets to determine where to manually truncate them using a mask:

In [18]:
'''for file in files:
    path = os.path.join(save_dir, file)

    timestamp, V, I, Temperatures, comments = load_dataset(path)
    n_therm = Temperatures.shape[1]

    T, run = pattern.search(file).groups()

    plt.figure(figsize = (12, 5))

    for i in range(n_therm):
        plt.plot(timestamp, Temperatures[:, i], label = f"Thermistor {i}")

    plt.xlabel("Time [s]")
    plt.ylabel("Temperature [°C]")
    
    plt.title(f"Time Period = {T} s, Run {run}")

    plt.grid(True)
    plt.legend(ncol = 4, fontsize = 9)
    plt.tight_layout()
    plt.show()'''

'for file in files:\n    path = os.path.join(save_dir, file)\n\n    timestamp, V, I, Temperatures, comments = load_dataset(path)\n    n_therm = Temperatures.shape[1]\n\n    T, run = pattern.search(file).groups()\n\n    plt.figure(figsize = (12, 5))\n\n    for i in range(n_therm):\n        plt.plot(timestamp, Temperatures[:, i], label = f"Thermistor {i}")\n\n    plt.xlabel("Time [s]")\n    plt.ylabel("Temperature [°C]")\n    \n    plt.title(f"Time Period = {T} s, Run {run}")\n\n    plt.grid(True)\n    plt.legend(ncol = 4, fontsize = 9)\n    plt.tight_layout()\n    plt.show()'

In [19]:
truncate_rules = {
    'brass_T10s_run1.csv':  lambda t: t > 500,
    'brass_T10s_run3.csv':  lambda t: t < 200,
    'brass_T15s_run1.csv':  lambda t: t > 500,
    'brass_T15s_run2.csv':  lambda t: t > 100,
    'brass_T20s_run1.csv':  lambda t: t > 200,
    'brass_T20s_run2.csv':  lambda t: t < 200,
    'brass_T20s_run3.csv':  lambda t: t < 200,
    'brass_T30s_run1.csv':  lambda t: t > 300,
    'brass_T30s_run2.csv':  lambda t: t > 100,
    'brass_T60s_run1.csv':  lambda t: t > 300,
    'brass_T60s_run2.csv':  lambda t: t < 450,
}

### **Fit Each Thermistor to a Sinusoid**

For each time period $\tau$ we will fit each thermistor to a sinusoid of the form:

$$
T(t) = T_0 + A\sin\left( \omega t + \phi \right), \quad \omega = \frac{2\pi}{\tau}
$$

to extract their amplitudes and phases.


In [20]:
def sinusoid(t, T0, A, phi, omega):
    return T0 + A * np.sin(omega * t + phi)

In [21]:
def fit_thermistor(time, temp, period):
    
    omega = 2 * np.pi / period

    # Initial guesses
    T0_guess = np.mean(temp)
    A_guess = 0.5 * (np.max(temp) - np.min(temp))
    phi_guess = 0.0

    popt, pcov = curve_fit(
        lambda t, T0, A, phi: sinusoid(t, T0, A, phi, omega),
        time,
        temp,
        p0 = [T0_guess, A_guess, phi_guess]
    )
    T0, A, phi = popt
    
    # Enforce A ≥ 0
    if A < 0:
        A = -A
        phi = phi + np.pi
        
    return T0, A, phi, pcov

### **Compute Complex Amplitude**

Using our extracted amplitudes and phases we define a complex amplitude:

$$
\tilde T = Ae^{i\phi}
$$

A single complex number now contains both the phase and amplitude information. We will now define a function to process each file, returning a dictionary of the complex amplitudes, their uncertainties and other metadata for each thermistor:

In [22]:
def process_file(file):

    path = os.path.join(save_dir, file)
    timestamp, V, I, Temps, comments = load_dataset(path)

    # Extract time period and run number from filename
    T_str, run_str = pattern.search(file).groups()
    period = float(T_str)

    # Apply truncation mask to remove transients
    mask = truncate_rules[file](timestamp)
    t_fit = timestamp[mask]
    temps_fit = Temps[mask, :]

    n_therm = temps_fit.shape[1]

    # Fit all thermistors
    T0_list = []
    A_list = []
    phi_list = []
    cov_list = []

    for i in range(n_therm):
        T0_i, A_i, phi_i, pcov_i = fit_thermistor(t_fit, temps_fit[:, i], period)
        T0_list.append(T0_i)
        A_list.append(A_i)
        phi_list.append(phi_i)
        cov_list.append(pcov_i)

    T0_arr = np.array(T0_list)
    A_arr = np.array(A_list)
    phi_arr = np.array(phi_list)

    # Complex amplitudes
    T_complex = A_arr * np.exp(1j * phi_arr)

    return period, {
        "file": file,
        "run": int(run_str),
        "t_fit": t_fit,
        "temps_fit": temps_fit,
        "T0": T0_arr,
        "A": A_arr,
        "phi": phi_arr,
        "cov": cov_list,
        "T_complex": T_complex
    }

In [23]:
raw_results_by_period = defaultdict(list)

for file in files:
    period, run_result = process_file(file)
    raw_results_by_period[period].append(run_result)

### **Compute Dimentionless Amplitude Ratio**

For each thermistor $i = 0, 1, 2, \dots , 7$ we normalise each run by the complex amplitude of thermistor 0 to form the dimensionless ratio:

$$
R_i = \frac{\tilde T_i}{\tilde T_0} = \frac{A_i}{A_0}e^{i(\phi_i - \phi_0)}.
$$

We then average this ratio across the runs for each time period to obtain an estimate of the statistical uncertainty:


In [24]:
summary_by_period = {}

for period, run_list in raw_results_by_period.items():
    # Stack complex amplitudes: shape (n_runs, n_therm)
    T_complex_runs = np.vstack([r["T_complex"] for r in run_list])
    n_runs, n_therm = T_complex_runs.shape

    # Compute ratios for each run: R_i^(r) = T_i^(r) / T_0^(r)
    R_runs = T_complex_runs / T_complex_runs[:, [0]]  # normalise each row by its first element

    # Mean and std across runs
    T_complex_mean = np.mean(T_complex_runs, axis=0)
    T_complex_std = np.std(T_complex_runs, axis=0, ddof=1)

    R_mean = np.mean(R_runs, axis=0)
    R_std = np.std(R_runs, axis=0, ddof=1)

    summary_by_period[period] = {
        "n_runs": n_runs,
        "T_complex_runs": T_complex_runs,
        "T_complex_mean": T_complex_mean,
        "T_complex_std": T_complex_std,
        "R_runs": R_runs,
        "R_mean": R_mean,
        "R_std": R_std,
    }

### **Inspect R_mean and R_std for each period**

for period in sorted(summary_by_period.keys()):
    R_mean = summary_by_period[period]["R_mean"]
    R_std = summary_by_period[period]["R_std"]
    n_runs = summary_by_period[period]["n_runs"]

    print(f"\n==============================")
    print(f"      Period T = {period:.0f} s")
    print(f"      Runs: {n_runs}")
    print("==============================\n")

    print("Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg(R_mean) [rad]      |R_std|")
    print("-------------------------------------------------------------------------------------------")

    for i, R in enumerate(R_mean):
        R_abs = np.abs(R)
        R_phase = np.angle(R)
        R_err = np.abs(R_std[i])  # magnitude of complex std

        print(f"{i:5d} | {R.real: .4e}   {R.imag: .4e}   {R_abs: .4e}        {R_phase: .4f}           {R_err: .4e}")



      Period T = 10 s
      Runs: 2

Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg(R_mean) [rad]      |R_std|
-------------------------------------------------------------------------------------------
    0 |  1.0000e+00   -5.5623e-18    1.0000e+00        -0.0000            1.6561e-17
    1 |  5.2960e-01   -2.9330e-01    6.0540e-01        -0.5058            1.3293e-03
    2 |  1.7991e-01   -3.1476e-01    3.6255e-01        -1.0515            9.2084e-04
    3 |  2.5431e-02   -2.3212e-01    2.3351e-01        -1.4617            8.4281e-04
    4 | -5.4237e-02   -1.2412e-01    1.3545e-01        -1.9828            7.8434e-04
    5 | -6.6599e-02   -4.6677e-02    8.1327e-02        -2.5303            8.5393e-04
    6 | -5.4936e-02   -2.6021e-04    5.4936e-02        -3.1369            8.3818e-04
    7 | -4.3676e-02    2.3842e-02    4.9760e-02         2.6419            9.1978e-04

      Period T = 15 s
      Runs: 2

Therm |     Re(R_mean)       Im(R_mean)      |R_mean|      arg

In [25]:
### Theoretical models for complex ratio R_i = T̃_i / T̃_0

def R_exp_model(D, x, omega):
    """Exponential thermal-wave model (one-way propagation)."""
    beta = (1 + 1j) * np.sqrt(omega / (2 * D))
    return np.exp(-beta * (x - x[0]))


def R_cosh_model(D, x, omega, L):
    """Full cosh model including reflection at non-conductive end."""
    beta = (1 + 1j) * np.sqrt(omega / (2 * D))
    # Normalised ratio:
    return np.cosh(beta * (L - x)) / np.cosh(beta * (L - x[0]))


In [26]:
from scipy.optimize import least_squares

def fit_model(R_mean, R_runs, x, omega, model, L=None):
    """
    Fit either the exponential ('exp') or cosh ('cosh') model
    to complex ratio data R_mean, using run-to-run scatter in
    R_runs to estimate uncertainties.
    """
    R_mean = np.asarray(R_mean)
    R_runs = np.asarray(R_runs)

    n_therm = len(x)
    fit_idx = np.arange(1, n_therm)  # skip thermistor 0 (R = 1 by definition)

    # --- empirical uncertainties from run-to-run scatter ---
    sigma_real = np.std(R_runs.real, axis=0, ddof=1)
    sigma_imag = np.std(R_runs.imag, axis=0, ddof=1)

    # ensure they're at least 1D arrays
    sigma_real = np.atleast_1d(sigma_real)
    sigma_imag = np.atleast_1d(sigma_imag)

    # avoid zero/tiny sigma → prevents division by zero or huge weights
    min_sigma = 1e-6
    sigma_real = np.maximum(sigma_real, min_sigma)
    sigma_imag = np.maximum(sigma_imag, min_sigma)

    # --- choose model ---
    def model_func(D):
        if model == "exp":
            return R_exp_model(D, x, omega)
        elif model == "cosh":
            return R_cosh_model(D, x, omega, L)
        else:
            raise ValueError("Unknown model type")

    # --- residuals: real and imag parts weighted separately ---
    def residuals(logD):
        # handle scalar or array
        logD = np.atleast_1d(logD)
        D = np.exp(logD[0])  # D > 0

        R_model = model_func(D)
        r = R_mean - R_model

        res_real = (r.real / sigma_real)[fit_idx]
        res_imag = (r.imag / sigma_imag)[fit_idx]

        return np.concatenate([res_real, res_imag])

    # initial guess
    logD0 = np.log(1e-5)
    result = least_squares(residuals, x0=[logD0])

    D_fit = np.exp(result.x[0])
    res_final = result.fun              # residuals at best fit (already computed)
    chi2 = np.sum(res_final**2)

    n_params = 1
    n_data = res_final.size

    AIC = chi2 + 2 * n_params
    BIC = chi2 + n_params * np.log(n_data)

    return {
        "D_fit": D_fit,
        "chi2": chi2,
        "AIC": AIC,
        "BIC": BIC,
        "residuals": res_final,
        "result": result,
    }


In [27]:
L_mm = 41.0
L_m  = L_mm / 1000.0

x_mm = np.arange(8) * delta_d
x_m  = x_mm / 1000.0


In [28]:
fit_results = {}

for period, data in sorted(summary_by_period.items()):
    omega = 2 * np.pi / period

    R_mean = data["R_mean"]
    R_runs = data["R_runs"]

    fit_exp  = fit_model(R_mean, R_runs, x_m, omega, model="exp",  L=L_m)
    fit_cosh = fit_model(R_mean, R_runs, x_m, omega, model="cosh", L=L_m)

    fit_results[period] = {
        "exp":  fit_exp,
        "cosh": fit_cosh,
    }

    print(f"\n======================")
    print(f"  T = {period:.0f} s fits")
    print("======================")
    print(f"Exponential:  D = {fit_exp['D_fit']:.3e},  χ² = {fit_exp['chi2']:.3f}, "
          f"AIC = {fit_exp['AIC']:.2f}, BIC = {fit_exp['BIC']:.2f}")
    print(f"Cosh:         D = {fit_cosh['D_fit']:.3e},  χ² = {fit_cosh['chi2']:.3f}, "
          f"AIC = {fit_cosh['AIC']:.2f}, BIC = {fit_cosh['BIC']:.2f}")



  T = 10 s fits
Exponential:  D = 3.017e-05,  χ² = 4412.009, AIC = 4414.01, BIC = 4414.65
Cosh:         D = 3.023e-05,  χ² = 3671.810, AIC = 3673.81, BIC = 3674.45

  T = 15 s fits
Exponential:  D = 3.052e-05,  χ² = 906.883, AIC = 908.88, BIC = 909.52
Cosh:         D = 3.170e-05,  χ² = 1629.998, AIC = 1632.00, BIC = 1632.64

  T = 20 s fits
Exponential:  D = 3.049e-05,  χ² = 1867.430, AIC = 1869.43, BIC = 1870.07
Cosh:         D = 3.076e-05,  χ² = 937.677, AIC = 939.68, BIC = 940.32

  T = 30 s fits
Exponential:  D = 2.835e-05,  χ² = 3027343.103, AIC = 3027345.10, BIC = 3027345.74
Cosh:         D = 2.906e-05,  χ² = 548802.167, AIC = 548804.17, BIC = 548804.81

  T = 60 s fits
Exponential:  D = 2.899e-05,  χ² = 24738.926, AIC = 24740.93, BIC = 24741.57
Cosh:         D = 3.593e-05,  χ² = 7256.484, AIC = 7258.48, BIC = 7259.12
